In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix
# import modeling clasiifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

import nltk
import unicodedata
import re

# Acquire

In [2]:
dw = pd.read_csv('dw.csv')
dw.head()

,Unnamed: 0,repo,language,readme_contents
0,0,eccentricdevotion/TARDIS,Java,# TARDIS\n\nTARDIS is a Spigot / Paper plugin ...
1,1,kuralabs/reactive-core-doctor-who-web,JavaScript,==============================================...
2,2,fwallacephd/doctor-who,CSS,Notes about open source Doctor Who Project:\n\...
3,3,kuralabs/reactive-core-doctor-who-core,JavaScript,==============================================...
4,4,kuralabs/reactive-core-doctor-who-mobile,JavaScript,==============================================...


# Prepare

In [3]:
dw = pd.read_csv('clean_dw.csv')
dw.head()

,Unnamed: 0,Unnamed: 0.1,repo,language,readme_contents,cleaned_content,tokenized_content,stemmed_content,lemma_content,no_stopwords_stem,no_stopwords_lemma,all_clean_stem,all_clean_lemma
0,0,0,eccentricdevotion/TARDIS,Java,# TARDIS\n\nTARDIS is a Spigot / Paper plugin ...,tardis\n\ntardis is a spigot paper plugin th...,tardis\n\ntardis is a spigot paper plugin that...,tardi tardi is a spigot paper plugin that allo...,tardis tardis is a spigot paper plugin that al...,spigot paper plugin allows create lets time tr...,spigot paper plugin allows create lets time tr...,spigot paper plugin allows create lets time tr...,spigot paper plugin allows create lets time tr...
1,1,1,kuralabs/reactive-core-doctor-who-web,JavaScript,==============================================...,\ndoctor who web app reactive core architectu...,doctor who web app reactive core architecture ...,doctor who web app reactiv core architectur de...,doctor who web app reactive core architecture ...,web app reactive core architecture demo applic...,web app reactive core architecture demo applic...,web app reactive core architecture demo applic...,web app reactive core architecture demo applic...
2,2,2,fwallacephd/doctor-who,CSS,Notes about open source Doctor Who Project:\n\...,notes about open source doctor who project\n\n...,notes about open source doctor who project\n\n...,note about open sourc doctor who project thi s...,note about open source doctor who project this...,notes open source project site created open so...,notes open source project site created open so...,notes open source project site created open so...,notes open source project site created open so...
3,3,3,kuralabs/reactive-core-doctor-who-core,JavaScript,==============================================...,\ndoctor who reactive core reactive core arch...,doctor who reactive core reactive core archite...,doctor who reactiv core reactiv core architect...,doctor who reactive core reactive core archite...,reactive core reactive core architecture demo ...,reactive core reactive core architecture demo ...,reactive core reactive core architecture demo ...,reactive core reactive core architecture demo ...
4,4,4,kuralabs/reactive-core-doctor-who-mobile,JavaScript,==============================================...,\ndoctor who mobile app reactive core archite...,doctor who mobile app reactive core architectu...,doctor who mobil app reactiv core architectur ...,doctor who mobile app reactive core architectu...,mobile app reactive core architecture demo app...,mobile app reactive core architecture demo app...,mobile app reactive core architecture demo app...,mobile app reactive core architecture demo app...


# Let's Model!

In [4]:
# We'll use this split function later to create in-sample and out-of-sample datasets for modeling
def split(df, stratify_by=None):
    """
    3 way split for train, validate, and test datasets
    To stratify, send in a column name
    """
    
    train, test = train_test_split(df, test_size=.2, random_state=123, stratify=df[stratify_by])
    
    train, validate = train_test_split(train, test_size=.3, random_state=123, stratify=train[stratify_by])
    
    return train, validate, test

In [5]:
def split_top_4(dw):
    '''create the top 4 most used languages
    split these into train test and validate'''
    # create top 4
    top_4_characters = dw.language.value_counts().index[0:4]
    top_4 = dw[dw.language.isin(top_4_characters)]
    # split into train, validate, test
    train, validate, test = split(top_4, 'language')
    return train, validate, test

In [6]:
train, validate, test = split_top_4(dw)
train.head()

,Unnamed: 0,Unnamed: 0.1,repo,language,readme_contents,cleaned_content,tokenized_content,stemmed_content,lemma_content,no_stopwords_stem,no_stopwords_lemma,all_clean_stem,all_clean_lemma
21,24,24,MCS-Kaijin/WhoScript,Python,# WhoScript\nDoctor Who-themed esoteric progra...,whoscript\ndoctor whothemed esoteric programm...,whoscript\ndoctor whothemed esoteric programmi...,whoscript doctor whothem esoter program langua...,whoscript doctor whothemed esoteric programmin...,whoscript whothemed esoteric programming langu...,whoscript whothemed esoteric programming langu...,whoscript whothemed esoteric programming langu...,whoscript whothemed esoteric programming langu...
58,114,114,mollytb/DoctorWhoNoDoubleClick,JavaScript,This project was bootstrapped with [Create Rea...,this project was bootstrapped with create reac...,this project was bootstrapped with create reac...,thi project wa bootstrap with creat react apph...,this project wa bootstrapped with create react...,project bootstrapped create react apphttpsgith...,project bootstrapped create react apphttpsgith...,project bootstrapped create react apphttpsgith...,project bootstrapped create react apphttpsgith...
84,194,194,Splodge97/DoctorWhoClubPenguinTresureHunt,HTML,# DoctorWhoClubPenguinTresureHunt\nDoctor Who ...,doctorwhoclubpenguintresurehunt\ndoctor who t...,doctorwhoclubpenguintresurehunt\ndoctor who th...,doctorwhoclubpenguintresurehunt doctor who the...,doctorwhoclubpenguintresurehunt doctor who the...,doctorwhoclubpenguintresurehunt themed treasur...,doctorwhoclubpenguintresurehunt themed treasur...,doctorwhoclubpenguintresurehunt themed treasur...,doctorwhoclubpenguintresurehunt themed treasur...
70,142,142,aseitz94/TriviaGame,HTML,# TriviaGame\nDoctor Who Basic Trivia Game usi...,triviagame\ndoctor who basic trivia game usin...,triviagame\ndoctor who basic trivia game using...,triviagam doctor who basic trivia game use my ...,triviagame doctor who basic trivia game using ...,triviagame basic trivia game using current kno...,triviagame basic trivia game using current kno...,triviagame basic trivia game using current kno...,triviagame basic trivia game using current kno...
69,140,140,frabcus/kill-the-moon,Python,"""Kill the Moon"" electricity use\n=============...",kill the moon electricity use\n\n\nwarning con...,kill the moon electricity use\n\n\nwarning con...,kill the moon electr use warn contain spoiler ...,kill the moon electricity use warning contains...,kill moon electricity warning contains spoiler...,kill moon electricity warning contains spoiler...,kill moon electricity warning contains spoiler...,kill moon electricity warning contains spoiler...


In [7]:
def split_into_x_y(train, validate, test):
    '''seperates train, validate, and test
    into X and y variables'''
    # Setup our X variables
    X_train = train.all_clean_lemma
    X_validate = validate.all_clean_lemma
    X_test = test.all_clean_lemma
    # set up our y variables
    y_train = train.language
    y_validate = validate.language
    y_test = test.language
    # return them
    return X_train, X_validate, X_test, y_train, y_validate, y_test

X_train, X_validate, X_test, y_train, y_validate, y_test = split_into_x_y(train, validate, test)
X_train.head()

21    whoscript whothemed esoteric programming langu...
58    project bootstrapped create react apphttpsgith...
84    doctorwhoclubpenguintresurehunt themed treasur...
70    triviagame basic trivia game using current kno...
69    kill moon electricity warning contains spoiler...
Name: all_clean_lemma, dtype: object

In [8]:
y_train.head()

21        Python
58    JavaScript
84          HTML
70          HTML
69        Python
Name: language, dtype: object

In [9]:
def vectorize_training(X_train, X_validate, X_test):
    # Create the tfidf vectorizer object
    tfidf = TfidfVectorizer()
    # Fit on the training data
    tfidf.fit(X_train)
    # the the training data
    X_train_vectorized = tfidf.transform(X_train)
    X_validate_vectorized = tfidf.transform(X_validate)
    X_test_vectorized = tfidf.transform(X_test)
    # return them
    return X_train_vectorized, X_validate_vectorized, X_test_vectorized
X_train_vectorized, X_validate_vectorized, X_test_vectorized = vectorize_training(X_train, X_validate, X_test)

In [10]:
X_train_vectorized.todense().shape
# so pretty much every single word has a column

(44, 3503)

# Logistic Regression Model

In [11]:
lm = LogisticRegression(random_state=123)
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
lm.fit(X_train_vectorized, y_train)

LogisticRegression(random_state=123)

In [12]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))

In [13]:
# make your predictions
train['predicted'] = lm.predict(X_train_vectorized)
validate["predicted"] = lm.predict(X_validate_vectorized)
test['predicted'] = lm.predict(X_test_vectorized)

In [14]:
# Train Accuracy
logit_trian = (train.actual == train.predicted).mean()

In [15]:
# Validate Accuracy
    # AKA Out of Sample Accuracy
logit_validate = (validate.actual == validate.predicted).mean()

In [16]:
def logistic_regression(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    '''makes the model
    fit the model
    make actual and predicted outcomes
    test accuracy on train and validate
    return the accuracy readings'''
    lm = LogisticRegression(random_state=123)
    # Fit a model using only these specified features
    lm.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = lm.predict(X_train_vectorized)
    validate["predicted"] = lm.predict(X_validate_vectorized)
    test['predicted'] = lm.predict(X_test_vectorized)
    # Train Accuracy
    train = (train.actual == train.predicted).mean()
    # Validate Accuracy
    validate = (validate.actual == validate.predicted).mean()
    print(f"The accuracy on the train data set is: \n", round(train,3))
    print(f"The accuracy on the validate data set is: \n", round(validate,3))
logistic_regression(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)

The accuracy on the train data set is: 
 0.955
The accuracy on the validate data set is: 
 0.421


- HOLY BAD VALIDATE RUN BATMAN

___

# Balanced Logistic Regression Model

In [17]:
lm2 = LogisticRegression(random_state=123, class_weight='balanced')
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
lm2.fit(X_train_vectorized, y_train)

LogisticRegression(class_weight='balanced', random_state=123)

In [18]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))
# make your predictions
train['predicted'] = lm2.predict(X_train_vectorized)
validate["predicted"] = lm2.predict(X_validate_vectorized)
test['predicted'] = lm2.predict(X_test_vectorized)

In [19]:
logit2_train = lm2.score(X_train_vectorized, y_train)
print(f"The accuracy on the train data set is: \n", round(logit2_train,3))  

The accuracy on the train data set is: 
 0.977


In [20]:
logit2_validate = lm2.score(X_validate_vectorized, y_validate)
print(f"The accuracy on the validate data set is: \n", round(logit2_validate,3))

The accuracy on the validate data set is: 
 0.526


In [21]:
def balanced_log_regression(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    '''makes the model
    fit the model
    make actual and predicted outcomes
    test accuracy on train and validate
    return the accuracy readings'''
    lm2 = LogisticRegression(random_state=123, class_weight='balanced')
    # Fit a model using only these specified features
    lm2.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = lm2.predict(X_train_vectorized)
    validate["predicted"] = lm2.predict(X_validate_vectorized)
    test['predicted'] = lm2.predict(X_test_vectorized)
    # Train Accuracy
    train = lm2.score(X_train_vectorized, y_train)
    # Validate Accuracy
    validate = lm2.score(X_validate_vectorized, y_validate)
    print(f"The accuracy on the train data set is: \n", round(train,3))
    print(f"The accuracy on the validate data set is: \n", round(validate,3))
balanced_log_regression(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)

The accuracy on the train data set is: 
 0.977
The accuracy on the validate data set is: 
 0.526


- Once again not the best run on validate

___

# KNN Model

In [22]:
knn = KNeighborsClassifier()
# Fit a model using only these specified features
knn.fit(X_train_vectorized, y_train)

KNeighborsClassifier()

In [23]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))
# make your predictions
train['predicted'] = knn.predict(X_train_vectorized)
validate["predicted"] = knn.predict(X_validate_vectorized)
test['predicted'] = knn.predict(X_test_vectorized)

In [24]:
knn_train = knn.score(X_train_vectorized, y_train)
knn_validate = knn.score(X_validate_vectorized, y_validate)
knn_train, knn_validate

(0.5681818181818182, 0.5789473684210527)

In [25]:
def knn_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    '''makes the model
    fit the model
    make actual and predicted outcomes
    test accuracy on train and validate
    return the accuracy readings'''
    knn = KNeighborsClassifier()
    # Fit a model using only these specified features
    knn.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = knn.predict(X_train_vectorized)
    validate["predicted"] = knn.predict(X_validate_vectorized)
    test['predicted'] = knn.predict(X_test_vectorized)
    # train accuracy
    train = knn.score(X_train_vectorized, y_train)
    # validate accuracy
    validate = knn.score(X_validate_vectorized, y_validate)
    print(f"The accuracy on the train data set is: \n", round(train,3))
    print(f"The accuracy on the validate data set is: \n", round(validate,3))
knn_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)

The accuracy on the train data set is: 
 0.568
The accuracy on the validate data set is: 
 0.579


- Still bad

___

# Decision Tree

In [26]:
clf1 = DecisionTreeClassifier(max_depth=3)
# Fit a model using only these specified features
clf1.fit(X_train_vectorized, y_train)

DecisionTreeClassifier(max_depth=3)

In [27]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))
# make your predictions
train['predicted'] = clf1.predict(X_train_vectorized)
validate["predicted"] = clf1.predict(X_validate_vectorized)
test['predicted'] = clf1.predict(X_test_vectorized)

In [28]:
dc_train = clf1.score(X_train_vectorized, y_train)
dc_validate = clf1.score(X_validate_vectorized, y_validate)
dc_train, dc_validate

(0.5227272727272727, 0.3684210526315789)

In [29]:
def decision_tree_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    '''makes the model
    fit the model
    make actual and predicted outcomes
    test accuracy on train and validate
    return the accuracy readings'''
    clf1 = DecisionTreeClassifier(max_depth=3)
    # Fit a model using only these specified features
    clf1.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = clf1.predict(X_train_vectorized)
    validate["predicted"] = clf1.predict(X_validate_vectorized)
    test['predicted'] = clf1.predict(X_test_vectorized)
    # train accuracy
    train = clf1.score(X_train_vectorized, y_train)
    # validate accuracy
    validate = clf1.score(X_validate_vectorized, y_validate)
    print(f"The accuracy on the train data set is: \n", round(train,3))
    print(f"The accuracy on the validate data set is: \n", round(validate,3))
decision_tree_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)

The accuracy on the train data set is: 
 0.523
The accuracy on the validate data set is: 
 0.316


- Still not great

___

# Random Forest Classifier

In [30]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,
                            max_depth=3, 
                            random_state=123)
# Fit a model using only these specified features
rf.fit(X_train_vectorized, y_train)

RandomForestClassifier(max_depth=3, min_samples_leaf=3, random_state=123)

In [31]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))
# make your predictions
train['predicted'] = rf.predict(X_train_vectorized)
validate["predicted"] = rf.predict(X_validate_vectorized)
test['predicted'] = rf.predict(X_test_vectorized)

In [32]:
rf_train = rf.score(X_train_vectorized, y_train)
rf_validate = rf.score(X_validate_vectorized, y_validate)
rf_train, rf_validate

(0.4318181818181818, 0.42105263157894735)

In [33]:
def random_forest_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    rf = RandomForestClassifier(bootstrap=True, 
                                class_weight=None, 
                                criterion='gini',
                                min_samples_leaf=3,
                                n_estimators=100,
                                max_depth=3, 
                                random_state=123)
    # Fit a model using only these specified features
    rf.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = rf.predict(X_train_vectorized)
    validate["predicted"] = rf.predict(X_validate_vectorized)
    test['predicted'] = rf.predict(X_test_vectorized)
    # train accuracy
    train = rf.score(X_train_vectorized, y_train)
    # validate_accuracy
    validate = rf.score(X_validate_vectorized, y_validate)
    print(f"The accuracy on the train data set is: \n", round(train,3))
    print(f"The accuracy on the validate data set is: \n", round(validate,3))
random_forest_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)


The accuracy on the train data set is: 
 0.432
The accuracy on the validate data set is: 
 0.421


___

# Ridge Classifier Model

In [34]:
clf2 = RidgeClassifier(random_state=123)
# Fit a model using only these specified features
# logit.fit(X_train[["age", "pclass", "fare"]], y_train)
clf2.fit(X_train_vectorized, y_train)

RidgeClassifier(random_state=123)

In [35]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))
# make your predictions
train['predicted'] = clf2.predict(X_train_vectorized)
validate["predicted"] = clf2.predict(X_validate_vectorized)
test['predicted'] = clf2.predict(X_test_vectorized)

In [36]:
ridge_train = clf2.score(X_train_vectorized, y_train)
ridge_validate = clf2.score(X_validate_vectorized, y_validate)
ridge_train, ridge_validate

(0.9772727272727273, 0.47368421052631576)

In [37]:
def ridge_class_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    '''makes the model
    fit the model
    make actual and predicted outcomes
    test accuracy on train and validate
    return the accuracy readings'''
    clf2 = DecisionTreeClassifier(max_depth=3)
    # Fit a model using only these specified features
    clf2.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = clf2.predict(X_train_vectorized)
    validate["predicted"] = clf2.predict(X_validate_vectorized)
    test['predicted'] = clf2.predict(X_test_vectorized)
    # train accuracy
    train = clf2.score(X_train_vectorized, y_train)
    # validate accuracy
    validate = clf2.score(X_validate_vectorized, y_validate)
    print(f"The accuracy on the train data set is: \n", round(train,3))
    print(f"The accuracy on the validate data set is: \n", round(validate,3))
ridge_class_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)

The accuracy on the train data set is: 
 0.523
The accuracy on the validate data set is: 
 0.368


- still not great

___

# SGDClassifier Model

In [38]:
clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123)
# Fit a model using only these specified features
clf3.fit(X_train_vectorized, y_train)

SGDClassifier(random_state=123)

In [39]:
# make the actual outcomes
train = pd.DataFrame(dict(actual=y_train))
validate = pd.DataFrame(dict(actual=y_validate))
test = pd.DataFrame(dict(actual=y_test))
# make your predictions
train['predicted'] = clf3.predict(X_train_vectorized)
validate["predicted"] = clf3.predict(X_validate_vectorized)
test['predicted'] = clf3.predict(X_test_vectorized)

In [40]:
sgd_train = clf3.score(X_train_vectorized, y_train)
sgd_validate = clf3.score(X_validate_vectorized, y_validate)
sgd_train, sgd_validate 

(0.9772727272727273, 0.5789473684210527)

In [41]:
def sgd_class_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    '''makes the model
    fit the model
    make actual and predicted outcomes
    test accuracy on train and validate
    return the accuracy readings'''
    clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123)
    # Fit a model using only these specified features
    clf3.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = clf3.predict(X_train_vectorized)
    validate["predicted"] = clf3.predict(X_validate_vectorized)
    test['predicted'] = clf3.predict(X_test_vectorized)
    # train accuracy
    train = clf3.score(X_train_vectorized, y_train)
    # validate accuracy
    validate = clf3.score(X_validate_vectorized, y_validate)
    print(f"The accuracy on the train data set is: \n", round(train,3))
    print(f"The accuracy on the validate data set is: \n", round(validate,3))
sgd_class_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)

The accuracy on the train data set is: 
 0.977
The accuracy on the validate data set is: 
 0.579


___

In [42]:
print(f"Logistic Regression Model Train: \n", round(logit_trian,3))
print(f"Logistic Regrssion Model Validate: \n", round(logit_validate,3))
print("__________________________________________________________")
print(f"Logistic Regression Balanced Model Train: \n", round(logit2_train,3))
print(f"Logistic Regrssion Balanced Model Validate: \n", round(logit2_validate,3))
print("__________________________________________________________")
print(f"KNN Model Train: \n", round(knn_train,3))
print(f"KNN Model Validate: \n", round(knn_validate,3))
print("__________________________________________________________")
print(f"Decision Tree Model Train: \n", round(dc_train,3))
print(f"Decision Tree Model Validate: \n", round(dc_validate,3))
print("__________________________________________________________")
print(f"Random Forest Model Train: \n", round(rf_train,3))
print(f"Random Forest Model Validate: \n", round(rf_validate,3))
print("__________________________________________________________")
print(f"Ridge Classifier Model Train: \n", round(ridge_train,3))
print(f"Ridge Classifier Model Validate: \n", round(ridge_validate,3))
print("__________________________________________________________")
print(f"SGDClassifier Model Train: \n", round(sgd_train,3))
print(f"SGDClassifier Model Validate: \n", round(sgd_validate,3))
print("__________________________________________________________")

Logistic Regression Model Train: 
 0.955
Logistic Regrssion Model Validate: 
 0.421
__________________________________________________________
Logistic Regression Balanced Model Train: 
 0.977
Logistic Regrssion Balanced Model Validate: 
 0.526
__________________________________________________________
KNN Model Train: 
 0.568
KNN Model Validate: 
 0.579
__________________________________________________________
Decision Tree Model Train: 
 0.523
Decision Tree Model Validate: 
 0.368
__________________________________________________________
Random Forest Model Train: 
 0.432
Random Forest Model Validate: 
 0.421
__________________________________________________________
Ridge Classifier Model Train: 
 0.977
Ridge Classifier Model Validate: 
 0.474
__________________________________________________________
SGDClassifier Model Train: 
 0.977
SGDClassifier Model Validate: 
 0.579
__________________________________________________________


___

# Best Model

### SGDClassifier is the best model as of current

In [43]:
sgd_train = clf3.score(X_train_vectorized, y_train)
sgd_validate = clf3.score(X_validate_vectorized, y_validate)
sgd_test = clf3.score(X_test_vectorized, y_test)
sgd_train

0.9772727272727273

In [48]:
def sgd_best_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized):
    '''makes the model
    fit the model
    make actual and predicted outcomes
    test accuracy on train and validate
    return the accuracy readings'''
    clf3 = SGDClassifier(max_iter=1000, tol=1e-3, random_state=123)
    # Fit a model using only these specified features
    clf3.fit(X_train_vectorized, y_train)
    # make the actual outcomes
    train = pd.DataFrame(dict(actual=y_train))
    validate = pd.DataFrame(dict(actual=y_validate))
    test = pd.DataFrame(dict(actual=y_test))
    # make your predictions
    train['predicted'] = clf3.predict(X_train_vectorized)
    validate["predicted"] = clf3.predict(X_validate_vectorized)
    test['predicted'] = clf3.predict(X_test_vectorized)
    # train accuracy
    train = clf3.score(X_train_vectorized, y_train)
    # validate accuracy
    validate = clf3.score(X_validate_vectorized, y_validate)
    # test accuracy
    test = clf3.score(X_test_vectorized, y_test)
    print(f"SGDClassifier Model Train: \n", round(train,3))
    print(f"SGDClassifier Model Validate: \n", round(validate,3))
    print(f"SGDClassifier Model Test: \n", round(test,3))
sgd_best_model(y_train, y_validate, y_test, X_train_vectorized, X_validate_vectorized, X_test_vectorized)

SGDClassifier Model Train: 
 0.977
SGDClassifier Model Validate: 
 0.579
SGDClassifier Model Test: 
 0.438
